# Flight hours and code


- obtain the minimum flight hours between regions from Tequila-API
- get the iata code of all the airports in a country's captial 

In [4]:
import requests
import numpy as np
import pandas as pd

### Example usage of Tequila

In [2]:
url= \
"https://tequila-api.kiwi.com/v2/search?fly_from=JFK&fly_to=IAD&\
date_from=01%2F5%2F2021&date_to=05%2F12%2F2021&\
&adults=1&selected_cabins=C\
&only_working_days=false&only_weekends=false&partner_market=us\
&vehicle_type=aircraft&limit=500"


In [181]:
''' change orign and destination '''
origin='JFK'
destination='BOF'

# location in URL string 
i_o=url.find('fly_from=')+9
i_t=url.find('fly_to=')+7
url=url[:i_o]+origin+url[i_o+3:]
url=url[:i_t]+destination+url[i_t+3:]

In [89]:
url

'https://tequila-api.kiwi.com/v2/search?fly_from=JFK&fly_to=IAD&date_from=01%2F5%2F2021&date_to=05%2F12%2F2021&&adults=1&selected_cabins=C&only_working_days=false&only_weekends=false&partner_market=us&vehicle_type=aircraft&limit=500'

In [183]:
z=requests.get(url, headers={'apikey':'h9SPJMYOmyBUKErHTUpHls71cZtqXqly'}
).json()

In [184]:
'data' in z.keys()

True

In [173]:
#len(z['data'])

In [175]:
# get the minimum and mean flight duration between origin and destination 
durations=[]
for i in range(len(z['data'])):
    #h,m=z['data'][i]['duration']
    #durations.append(int(h)+int(m[:-1])/60.)
    durations.append(z['data'][i]['duration']['departure']/3600)
durations=np.array(durations)
print(durations.mean())
print(durations.min())

1.6604938271604937
1.6


## Represent countrie locations by capitals

In [5]:
capitals=pd.read_csv("country-list.csv")
#https://github.com/icyrockcom/country-capitals/blob/master/data/country-list.csv

In [6]:
capitals

,country,capital,type
0,Abkhazia,Sukhumi,countryCapital
1,Afghanistan,Kabul,countryCapital
2,Akrotiri and Dhekelia,Episkopi Cantonment,countryCapital
3,Albania,Tirana,countryCapital
4,Algeria,Algiers,countryCapital
...,...,...,...
243,Wallis and Futuna,Mata-Utu,countryCapital
244,Western Sahara,El Aaiún,countryCapital
245,Yemen,Sanaá,countryCapital
246,Zambia,Lusaka,countryCapital


In [11]:
# list of countries in sim
countries=pd.read_csv("prevalance_419.csv")
countries=countries[~countries.location.str.startswith("US.")]
countries=countries.location.to_numpy()

In [12]:
# create country:capital dictiionary 
capital_dict={}
for c in countries:
    row=capitals[capitals.country==c]
    cc=list(row.capital.to_numpy())
    
    capital_dict[c]=cc
    

In [13]:
# countries without captitals
for i in capital_dict:
    if len(capital_dict[i])==0:
        print(i)

Cote d'Ivoire
Eswatini
Sao Tome and Principe
West Bank and Gaza


In [14]:
# put in data manually 
capital_dict["Cote d'Ivoire"]=['Yamoussoukro', 'Abidjan']
capital_dict['Eswatini']=['Mbabane','Lobamba']
capital_dict['Sao Tome and Principe']=['Sao Tome']
capital_dict['West Bank and Gaza']=['Jerusalem','Gaza']

In [58]:
# rename some captials 
capital_dict['US']=['Washington']
capital_dict['Bolivia']=['La Paz / El Alto']
capital_dict['South Africa']=['Johannesburg']
capital_dict['Belize']=['Belize City']
capital_dict['Cuba']=['Cienfuegos']

In [46]:
capital_dict

{'Afghanistan': ['Kabul'],
 'Albania': ['Tirana'],
 'Algeria': ['Algiers'],
 'Andorra': ['Andorra la Vella'],
 'Angola': ['Luanda'],
 'Antigua and Barbuda': ["St. John's"],
 'Argentina': ['Buenos Aires'],
 'Armenia': ['Yerevan'],
 'Australia': ['Canberra'],
 'Austria': ['Vienna'],
 'Azerbaijan': ['Baku'],
 'Bahamas': ['Nassau'],
 'Bahrain': ['Manama'],
 'Bangladesh': ['Dhaka'],
 'Barbados': ['Bridgetown'],
 'Belarus': ['Minsk'],
 'Belgium': ['Brussels'],
 'Belize': ['Belize City'],
 'Bolivia': ['La Paz / El Alto'],
 'Bosnia and Herzegovina': ['Sarajevo'],
 'Botswana': ['Gaborone'],
 'Brazil': ['Brasília'],
 'Bulgaria': ['Sofia'],
 'Burkina Faso': ['Ouagadougou'],
 'Burma': ['Naypyidaw'],
 'Cabo Verde': ['Praia'],
 'Cameroon': ['Yaoundé'],
 'Canada': ['Ottawa'],
 'Central African Republic': ['Bangui'],
 'Chile': ['Santiago'],
 'China': ['Beijing'],
 'Colombia': ['Bogotá'],
 'Comoros': ['Moroni'],
 'Congo (Brazzaville)': ['Brazzaville'],
 'Congo (Kinshasa)': ['Kinshasa'],
 'Costa Rica': 

In [10]:
# Alpha-2 code 
iso=pd.read_csv("iso_country_code.csv")
# https://gist.github.com/tadast/8827699

In [11]:
iso

,Name,Code
0,Afghanistan,AF
1,Ã…land Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS
...,...,...
244,Wallis and Futuna,WF
245,Western Sahara,EH
246,Yemen,YE
247,Zambia,ZM


In [48]:
# regions without code 
iso2=pd.DataFrame(columns=iso.columns)
for c in capital_dict:
    iso2=iso2.append(iso[iso.Name==c],ignore_index=True)
    if ((iso.Name==c).sum()==0):
        print(c)

Cote d'Ivoire
Kosovo
West Bank and Gaza


In [49]:
iso2.rename(columns={'Name':"Country",'Code':'iso_country'},inplace=True)

In [50]:
# put in manually 
iso2=iso2.append(pd.DataFrame({'Country':"Cote d'Ivoire",'iso_country':'CI'},index=[0]))
iso2=iso2.append(pd.DataFrame({'Country':"Kosovo",'iso_country':'XK'},index=[0]))
iso2=iso2.append(pd.DataFrame({'Country':"West Bank and Gaza",'iso_country':'PS'},index=[0]))

In [37]:
iso2

,Country,iso_country
0,Afghanistan,AF
1,Albania,AL
2,Algeria,DZ
3,Andorra,AD
4,Angola,AO
...,...,...
157,Zimbabwe,ZW
158,US,US
0,Cote d'Ivoire,CI
0,Kosovo,XK


## Represent captials by airports

In [7]:
airports=pd.read_csv("airports.csv")
#https://github.com/arangodb/example-datasets/blob/master/Airports/airports.csv 
#airports.dropna(subset=['iata_code']).to_csv("airports.csv")

In [52]:
airports=airports.merge(iso2,on='iso_country',how='inner')

### List of all IATA code for countries

In [53]:
airports=airports[['Country','municipality','iata_code']]
airports

,Country,municipality,iata_code
0,US,Key Largo,OCA
1,US,Pilot Station,PQS
2,US,Crested Butte,CSE
3,US,Johnson City,JCY
4,US,Palmer,PMX
...,...,...,...
8572,Tajikistan,Kurgan-Tyube,KQT
8573,Singapore,NaN,QPG
8574,Singapore,Western Water Catchment,TGA
8575,Singapore,Seletar,XSP


In [54]:
# capital corrections 
capital_dict['Bolivia']=['La Paz / El Alto']
capital_dict['Costa Rica']=['San José (Alajuela)']
capital_dict['Djibouti']=['Djibouti City']
capital_dict['El Salvador']=['San Salvador (San Luis Talpa)']
capital_dict['Israel']=['Tel Aviv']
capital_dict['Kazakhstan']=['Nur-Sultan']
capital_dict['Monaco']=['Fontvieille']
capital_dict['Panama']=['Panamá City']
capital_dict['Sao Tome and Principe']=['São Tomé']
capital_dict['Sri Lanka']=['Colombo']
capital_dict['Eswatini']=['Manzini','Mpaka']
capital_dict['Seychelles']=['Mahe Island']

### limit to the airports in capitals

In [55]:

airport=pd.DataFrame(columns=airports.columns)
for i in capital_dict:
    #print(i)
    for c in capital_dict[i]:
        cond=(airports.municipality==c) & (airports.Country==i)
        row=airports[cond]
        airport=airport.append(row,ignore_index=True)
        # no airport in capitals 
        if row.shape[0]==0:
            print(i,',',c)

Liechtenstein , Vaduz
San Marino , San Marino
West Bank and Gaza , Gaza


In [56]:
# create country:IATA corrospondences
iata={i:[] for i in capital_dict}
## remove non-existent airports
rm=['ZXT','TR7','ULC','NIC','HEX','DIA','PPJ','OSF','CKL','HPR','BYV',
'KLA','AZI','JRS','BOF','BZS', 'JPN','OCW']


for i in capital_dict:
    #print(i)
    for c in capital_dict[i]:
        cond=(airports.municipality==c) & (airports.Country==i)
        row=list(airports[cond].iata_code.to_numpy())

        for a in row:          
            if a not in rm:
                iata[i].append(a)

In [65]:
# correction for popular airports
iata['Switzerland']=['GVA','ZRH']
iata['Panama']=['PTY']
iata['Suriname']=['PBM']

### country exclusion

- countries too small to have airports: Liechtenstein, San Marino,Andorra,
 West Bank and Gaza?
- countries so small that even UN doesn't have data on them: Monaco


In [69]:
exclude=['Liechtenstein', 'San Marino','Andorra','West Bank and Gaza','Burma']
for k in exclude:
    if k in iata.keys():
        del iata[k]

In [71]:
iata

{'Afghanistan': ['KBL'],
 'Albania': ['TIA'],
 'Algeria': ['ALG'],
 'Angola': ['LAD'],
 'Antigua and Barbuda': ['ANU'],
 'Argentina': ['AEP', 'EZE'],
 'Armenia': ['EVN'],
 'Australia': ['CBR'],
 'Austria': ['VIE'],
 'Azerbaijan': ['GYD'],
 'Bahamas': ['NAS', 'PID'],
 'Bahrain': ['BAH'],
 'Bangladesh': ['DAC'],
 'Barbados': ['BGI'],
 'Belarus': ['MHP', 'MSQ'],
 'Belgium': ['BRU', 'CRL'],
 'Belize': ['TZA', 'BZE'],
 'Bolivia': ['LPB'],
 'Bosnia and Herzegovina': ['SJJ'],
 'Botswana': ['GBE'],
 'Brazil': ['BSB'],
 'Bulgaria': ['SOF'],
 'Burkina Faso': ['OUA'],
 'Cabo Verde': ['RAI'],
 'Cameroon': ['YAO', 'NSI'],
 'Canada': ['YOW', 'YRO'],
 'Central African Republic': ['BGF'],
 'Chile': ['SCL'],
 'China': ['PEK', 'PKX', 'NAY'],
 'Colombia': ['BOG'],
 'Comoros': ['HAH'],
 'Congo (Brazzaville)': ['BZV'],
 'Congo (Kinshasa)': ['FIH'],
 'Costa Rica': ['SJO'],
 "Cote d'Ivoire": ['ASK', 'ABJ'],
 'Croatia': ['ZAG'],
 'Cuba': ['CFG'],
 'Cyprus': ['ECN'],
 'Czechia': ['PRG'],
 'Denmark': ['CPH', 'R

In [ ]:
with open('country_iata.pkl','wb') as f:
    pickle.dump(iata,f)